In [1]:
import pandas as pd
import numpy as np
from symspellpy import SymSpell, Verbosity
from symspellpy.helpers import to_similarity

import pprint as pp

### Read data

Reading in input data directly from the homework assignment excel file. Name changed to 'input.xlsx'.

In [2]:
data = pd.read_excel('input.xlsx')

In [3]:
data.head()

,Input,Output
0,paypal * US - Doordash,"{""brands"":[{""1"":""paypal"", ""2"":""doordash""}], ""s..."
1,payal * US NY - Doodash,"{""brands"":[{""1"":""paypal"", ""2"":""doordash""}], ""s..."
2,sq* NJ SEAMLSS 2017777777,"{""brands"":[{""1"":""sq"", ""2"":""seamless""}], ""state..."
3,SEAMLSS MCd,"{""brands"":[{""1"":""seamless"", ""2"":""mcdonalds""}],..."
4,McDOnalds UBEREATS,"{""brands"":[{""1"":""mcdonalds"", ""2"":""ubereats""}],..."


Inspection of the given inputs shows state and country names are not misspelled. A simple search yields text files of states abbreviations and 2-letter ISO country codes to match tokens against. 

Importing list of states and country codes:

In [4]:
#load list of states
with open('states.txt') as f:
    states = f.readlines()
    
states = [x.rstrip().lower() for x in states] 
#load list of 2-letter country codes
with open('ISO_country_codes.txt') as f:
    countries = f.readlines()

countries = [x.rstrip().split(';')[1].lower() for x in countries] 
    

In [17]:
print(states)

['ak', 'al', 'az', 'ar', 'ca', 'co', 'ct', 'de', 'dc', 'fl', 'ga', 'hi', 'id', 'il', 'in', 'ia', 'ks', 'ky', 'la', 'me', 'mt', 'ne', 'nv', 'nh', 'nj', 'nm', 'ny', 'nc', 'nd', 'oh', 'ok', 'or', 'md', 'ma', 'mi', 'mn', 'ms', 'mo', 'pa', 'ri', 'sc', 'sd', 'tn', 'tx', 'ut', 'vt', 'va', 'wa', 'wv', 'wi', 'wy']


In [18]:
print(countries)

['af', 'ax', 'al', 'dz', 'as', 'ad', 'ao', 'ai', 'aq', 'ag', 'ar', 'am', 'aw', 'au', 'at', 'az', 'bs', 'bh', 'bd', 'bb', 'by', 'be', 'bz', 'bj', 'bm', 'bt', 'bo', 'bq', 'ba', 'bw', 'bv', 'br', 'io', 'bn', 'bg', 'bf', 'bi', 'kh', 'cm', 'ca', 'cv', 'ky', 'cf', 'td', 'cl', 'cn', 'cx', 'cc', 'co', 'km', 'cg', 'cd', 'ck', 'cr', 'ci', 'hr', 'cu', 'cw', 'cy', 'cz', 'dk', 'dj', 'dm', 'do', 'ec', 'eg', 'sv', 'gq', 'er', 'ee', 'et', 'fk', 'fo', 'fj', 'fi', 'fr', 'gf', 'pf', 'tf', 'ga', 'gm', 'ge', 'de', 'gh', 'gi', 'gr', 'gl', 'gd', 'gp', 'gu', 'gt', 'gg', 'gn', 'gw', 'gy', 'ht', 'hm', 'va', 'hn', 'hk', 'hu', 'is', 'in', 'id', 'ir', 'iq', 'ie', 'im', 'il', 'it', 'jm', 'jp', 'je', 'jo', 'kz', 'ke', 'ki', 'kp', 'kr', 'kw', 'kg', 'la', 'lv', 'lb', 'ls', 'lr', 'ly', 'li', 'lt', 'lu', 'mo', 'mk', 'mg', 'mw', 'my', 'mv', 'ml', 'mt', 'mh', 'mq', 'mr', 'mu', 'yt', 'mx', 'fm', 'md', 'mc', 'mn', 'me', 'ms', 'ma', 'mz', 'mm', 'na', 'nr', 'np', 'nl', 'nc', 'nz', 'ni', 'ne', 'ng', 'nu', 'nf', 'mp', 'no', 'om

### Extract brands, state, country, and phone number from input string

Data processing approach:

To yield expected output, each row is stripped of non-alphanumeric numbers, except for '.', which forms part of brand names. Input is then space-separated into tokens, which are identified as either phone numbers, states, countries, or brands. 
- Phone numbers must be all-numeric tokens of length 10.
- States are identified if belonging to state list.
- Country codes are identified if belongign ot country code list.
- Brand names are any tokens that do not fall into the above categories.

The out list maintains each processed row as a list of dictionaries. 

In [7]:
out= []

for row in data.Input:
    #lowercase and elminate special characters and punction except for '.' and spaces
    row = ''.join(c for c in row if c.isalnum() or c in ['.', ' '])
    
    #list of of space-separated tokens
    l = row.lower().strip().split()
    
    #output row
    d = {"brands":[{}], "state":"", "country": "", "ph no":""}
    brand_count = 1
    for token in l:
        
        #check if token is phone number, state, or country-- otherwise = brand
        if len(token)==10 and token.isnumeric():
            d['ph no'] = token
        elif token in states:
            d['state'] = token
        elif token in countries:
            d['country'] = token
        else:
            d['brands'][0][str(brand_count)] = token
            brand_count+=1
    
    out.append(d)        

Output is in correct format. Next, brand names must be corrected. 

In [8]:
pp.pprint(out)

[{'brands': [{'1': 'paypal', '2': 'doordash'}],
  'country': 'us',
  'ph no': '',
  'state': ''},
 {'brands': [{'1': 'payal', '2': 'doodash'}],
  'country': 'us',
  'ph no': '',
  'state': 'ny'},
 {'brands': [{'1': 'sq', '2': 'seamlss'}],
  'country': '',
  'ph no': '2017777777',
  'state': 'nj'},
 {'brands': [{'1': 'seamlss', '2': 'mcd'}],
  'country': '',
  'ph no': '',
  'state': ''},
 {'brands': [{'1': 'mcdonalds', '2': 'ubereats'}],
  'country': '',
  'ph no': '',
  'state': ''},
 {'brands': [{'1': 'giftcard'}], 'country': 'us', 'ph no': '', 'state': 'ca'},
 {'brands': [{'1': 'hotels.com', '2': 'expedia', '3': 'sheraton'}],
  'country': '',
  'ph no': '',
  'state': ''},
 {'brands': [{'1': 'chipotl', '2': 'postmates'}],
  'country': '',
  'ph no': '',
  'state': ''},
 {'brands': [{'1': 'paypal', '2': 'grubhub'}],
  'country': 'us',
  'ph no': '',
  'state': ''},
 {'brands': [{'1': 'payal', '2': 'ubereats'}],
  'country': 'us',
  'ph no': '',
  'state': 'ny'},
 {'brands': [{'1': 's

### Spelling correction and fuzzy string search

Load spell check library and brand dictionary. 

SymSpell performs spell-checking by loading a dictionary of terms and their respective frequencies for weight in predictions. When an out-of-dictionary term is encountered, it computes candidate terms from its dictionary based on how many edits are required to get from input word to in-dictionary terms, as well as in-dictionary term frequencies as likelihood of term. It uses the Symmetic Delete spelling correction algorithm and excels in speed. For more information about this algorithm and how it compares to others, see <a href="https://towardsdatascience.com/symspell-vs-bk-tree-100x-faster-fuzzy-string-search-spell-checking-c4f10d80a078">this post</a>.

Maximal edit distance chosen to be 6 here because it takes 6 edits to get from input token "mcd" to desired token "mcdonalds", the largest edit distance necessary for our inputs. Note, the more edits are necessary, the more computationally expensive it will be to spell-check words.

In [9]:
sym_spell = SymSpell(max_dictionary_edit_distance=6, prefix_length=7)
sym_spell.load_dictionary('brands_sep.txt', 0, 1, separator=";")


True

To show this a robust solution, the brand dictionary includes names of other popular tech brands to potentially correct misspellings to. All brand names are weighted equally.

Here is a preview of the dictionary:

In [10]:
sym_spell.words.items()

dict_items([('paypal', 2), ('doordash', 2), ('sq', 1), ('seamless', 1), ('mcdonalds', 1), ('ubereats', 1), ('hotels.com', 1), ('expedia', 1), ('sheraton', 1), ('chipotle', 1), ('postmates', 2), ('grubhub', 1), ('subway', 1), ('wendys', 1), ('giftcard', 1), ('booking.com', 2), ('priceline', 1), ('marriott', 1), ('chickfila', 1), ('Amazon', 1), ('Google', 1), ('Apple', 1), ('Microsoft', 1), ('Facebook', 1), ('Samsung', 1), ('Huawei', 1), ('WeChat', 1), ('YouTube', 1), ('Taobao', 1), ('IBM', 1), ('Tmall', 1), ('Intel', 1), ('Instagram', 1), ('accenture', 1), ('Oracle', 1), ('Cisco', 1), ('Netflix', 1), ('Alibaba.com', 1), ('SAP', 1), ('Uber', 1), ('NetEase', 1), ('JD.com', 1), ('Sony', 1), ('Midea', 1), ('Panasonic', 1), ('Airbnb', 1), ('Salesforce', 1), ('Nokia', 1), ('Gree', 1), ('Canon', 1), ('Adobe', 1), ('HP', 1), ('Baidu', 1), ('3M', 1), ('TSMC', 1), ('Cognizant', 1), ('Yahoo! Group', 1), ('eBay', 1), ('LinkedIn', 1), ('Haier', 1), ('Playstation', 1), ('Qualcomm', 1), ('Broadcom', 1

Here's an example of correcting the token "mcd" to "mcdonalds".

Choosing the corrected word simply from the one requiring the fewest edits can lead to errors given short tokens, such as "mcd", which could be entirely changed to a different in-dictionary token with fewer edits than the real target, "mcdonalds". 

Similarity measure is calculated using edit distance as (1 - (length / distance)).

In [11]:
suggestions = sym_spell.lookup('mcd', Verbosity,
                               max_edit_distance=6)

for sugg in suggestions:
    print('Candidate Term: {}, Edit Distance: {}, Similarity : {}'.format(sugg.term, sugg.distance, to_similarity(sugg.distance, len(sugg.term))))

Candidate Term: sq, Edit Distance: 3, Similarity : -0.5
Candidate Term: IBM, Edit Distance: 3, Similarity : 0.0
Candidate Term: SAP, Edit Distance: 3, Similarity : 0.0
Candidate Term: HP, Edit Distance: 3, Similarity : -0.5
Candidate Term: 3M, Edit Distance: 3, Similarity : -0.5
Candidate Term: HPE, Edit Distance: 3, Similarity : 0.0
Candidate Term: FIS, Edit Distance: 3, Similarity : 0.0
Candidate Term: HCL, Edit Distance: 3, Similarity : 0.0
Candidate Term: ZTE, Edit Distance: 3, Similarity : 0.0
Candidate Term: BOE, Edit Distance: 3, Similarity : 0.0
Candidate Term: CGI, Edit Distance: 3, Similarity : 0.0
Candidate Term: QVC, Edit Distance: 3, Similarity : 0.0
Candidate Term: Midea, Edit Distance: 4, Similarity : 0.19999999999999996
Candidate Term: Baidu, Edit Distance: 4, Similarity : 0.19999999999999996
Candidate Term: Nidec, Edit Distance: 4, Similarity : 0.19999999999999996
Candidate Term: Cisco, Edit Distance: 4, Similarity : 0.19999999999999996
Candidate Term: Tmall, Edit Dist

### Corecting brand names and final output

Checks each entry and each brand. If a brand is not in dictionary, it looks for candidate terms. 

We order the suggestions by similarity to original token and then fewest edits should there be a tie. 

In [12]:
for entry in out:
    for key,brand in entry['brands'][0].items():
        #if brand not in dictionary, spellcheck
        correct = sym_spell.lookup(brand, Verbosity.CLOSEST,
                               max_edit_distance=0)
        
        if not correct:
            suggestions = sym_spell.lookup(brand, Verbosity,
                               max_edit_distance=6)
            
            #if only one suggestion, choose it
            if len(suggestions)==1:
                entry['brands'][0][key] = suggestions[0].term
            
            #if more than one suggestion, order by similarity to original word, then no. edits
            else:
                corrections = []
                
                for sugg in suggestions:
                    corrections.append((sugg.term, sugg.distance, to_similarity(sugg.distance, len(sugg.term))))
                
                #already sorted by minimum edits, sort by similarity (descending)
                corrections = sorted(corrections, key=lambda x: -x[2])
                entry['brands'][0][key] = corrections[0][0]

### Inspect Output

Produced output = desired output 

In [13]:
for row in out:
    print(row)

{'brands': [{'1': 'paypal', '2': 'doordash'}], 'state': '', 'country': 'us', 'ph no': ''}
{'brands': [{'1': 'paypal', '2': 'doordash'}], 'state': 'ny', 'country': 'us', 'ph no': ''}
{'brands': [{'1': 'sq', '2': 'seamless'}], 'state': 'nj', 'country': '', 'ph no': '2017777777'}
{'brands': [{'1': 'seamless', '2': 'mcdonalds'}], 'state': '', 'country': '', 'ph no': ''}
{'brands': [{'1': 'mcdonalds', '2': 'ubereats'}], 'state': '', 'country': '', 'ph no': ''}
{'brands': [{'1': 'giftcard'}], 'state': 'ca', 'country': 'us', 'ph no': ''}
{'brands': [{'1': 'hotels.com', '2': 'expedia', '3': 'sheraton'}], 'state': '', 'country': '', 'ph no': ''}
{'brands': [{'1': 'chipotle', '2': 'postmates'}], 'state': '', 'country': '', 'ph no': ''}
{'brands': [{'1': 'paypal', '2': 'grubhub'}], 'state': '', 'country': 'us', 'ph no': ''}
{'brands': [{'1': 'paypal', '2': 'ubereats'}], 'state': 'ny', 'country': 'us', 'ph no': ''}
{'brands': [{'1': 'sq', '2': 'doordash'}], 'state': 'nj', 'country': '', 'ph no': '

In [14]:
for row in data.Output:
    print(row)

{"brands":[{"1":"paypal", "2":"doordash"}], "state":"", "country":"us","ph no":""}
{"brands":[{"1":"paypal", "2":"doordash"}], "state":"ny", "country":"us","ph no":""}
{"brands":[{"1":"sq", "2":"seamless"}], "state":"nj", "country":"","ph no":"2017777777"}
{"brands":[{"1":"seamless", "2":"mcdonalds"}], "state":"", "country":"","ph no":""}
{"brands":[{"1":"mcdonalds", "2":"ubereats"}], "state":"", "country":"","ph no":""}
{"brands":[{"1":"giftcard"}], "state":"ca", "country":"us","ph no":""}
{"brands":[{"1":"hotels.com", "2":"expedia", "3":"sheraton"}], "state":"", "country":"","ph no":""}
{"brands":[{"1":"chipotle", "2":"postmates"}], "state":"", "country":"","ph no":""}
{"brands":[{"1":"paypal", "2":"grubhub"}], "state":"", "country":"us","ph no":""}
{"brands":[{"1":"paypal", "2":"ubereats"}], "state":"ny", "country":"us","ph no":""}
{"brands":[{"1":"sq", "2":"doordash"}], "state":"nj", "country":"","ph no":"5745745745"}
{"brands":[{"1":"seamless", "2":"subway"}], "state":"", "country